In [34]:
from glacier_clustering.pipelines import merge_data
import pandas as pd
import numpy as np

In [260]:
%load_ext kedro.ipython
session.run()

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


[01/02/23 20:46:54] INFO     Kedro project clustering                                                ]8;id=761648;file://C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\framework\session\session.py\session.py]8;;\:]8;id=937924;file://C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\framework\session\session.py#340\340]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\fabia\AppData\Local\Temp\ipykernel_40356\1338477363.py:2 in <cell line: 2>              │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\fabia\\AppData\\Local\\Temp\\ipykernel_40356\\1338477363.py'                         │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\framework\session\session.py:343   │
│ in run                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KedroSessionError: A run has already been completed as part of the active KedroSession. KedroSession has a 1-1 
mapping with runs, and thus only one run should be executed per session.

# Data Loading
Loading the data from data source.

In [100]:
merge_data.create_pipeline().data_sets()

{
    'loaded_change',
    'merged_data',
    'change',
    'mass_balance',
    'loaded_state',
    'state',
    'glacier',
    'loaded_mass_balance',
    'loaded_glacier'
}

In [129]:
loaded_glacier = catalog.load("loaded_glacier")
loaded_change = catalog.load("loaded_change")
loaded_mass_balance = catalog.load("loaded_mass_balance")
loaded_state = catalog.load("loaded_state")

[01/01/23 22:19:22] INFO     Loading data from 'loaded_glacier' (ParquetDataSet)...             ]8;id=90621;file://C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=40270;file://C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'loaded_change' (ParquetDataSet)...              ]8;id=195254;file://C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=304523;file://C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'loaded_mass_balance' (ParquetDataSet)...        ]8;id=12774;file://C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=736795;file://C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'loaded_state' (ParquetDataSet)...               ]8;id=997053;file://C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=147717;file://C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

In [253]:
merged_data = loaded_glacier.merge(loaded_change, on="WGMS_ID", how="right")
merged_data = merged_data.merge(loaded_state, on=["WGMS_ID", "YEAR"], how="outer")
merged_data = merged_data.merge(loaded_mass_balance, on=["WGMS_ID", "YEAR"], how="outer")

test = merged_data.groupby("WGMS_ID")["AREA"].ffill().reset_index()

test.head()

,index,AREA
0,0,NaN
1,1,NaN
2,2,38.540001
3,3,38.540001
4,4,NaN


## EDA

In [ ]:
import seaborn as sns

# Abano Glacier
abano = merged_data.loc[merged_data["WGMS_ID"] == 767, :]
# Adler Glacier
adler = merged_data.loc[merged_data["WGMS_ID"] == 3801, :]

sns.lineplot(abano, y="THICKNESS_CHG", x="YEAR", title="ABANO")
sns.lineplot(adler, y="THICKNESS_CHG", x="YEAR", title="ADLER")

## Timeseries preparation

In [155]:
from tslearn.utils import to_time_series_dataset

# timeseries = to_time_series_dataset([i.t for i in merged_data.iterrows()])

features = ["AREA_CHANGE", "THICKNESS_CHG", "VOLUME_CHANGE"]

x = merged_data.WGMS_ID.factorize()[0]
y = merged_data.groupby(x).cumcount().values
vals = merged_data.loc[:, features].values
out_shp = (x.max()+1, y.max()+1, vals.shape[1])
out = np.full(out_shp, np.nan)
out[x,y] = vals

timeseries = to_time_series_dataset(out)

[01/01/23 23:14:11] WARNING  C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\utils\ ]8;id=143679;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py\warnings.py]8;;\:]8;id=555402;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py#109\109]8;;\
                             utils.py:149: DeprecationWarning:                                                     
                                                                                                                   
                             `np.float` is a deprecated alias for the builtin `float`. To silence                  
                             this warning, use `float` by itself. Doing this will not modify any                   
                             behavior and is safe. If you specifically wanted the numpy scalar                     
                             type, use `np.float64` here.                                                          
                             Deprecated in NumPy 1.20; for more details and guidance:                              
                             https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations                      
                                                                                                                   
                                                                                                                   

In [156]:
timeseries.shape

(213778, 19, 3)

In [254]:
t = None

for series in timeseries:
    mask = []
    for year in series:
        mask.append(np.isnan(year).all())
    if not all(mask):
        if t is None:
            t = [series]
        else:
            t = np.concatenate((t, [series]), axis=0)

In [255]:
t.shape

(210372, 19, 3)

In [256]:
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler
from tslearn.clustering import TimeSeriesKMeans


np.random.shuffle(timeseries)
# Keep only 50 time series
X_train = TimeSeriesScalerMeanVariance().fit_transform(t)
# Make time series shorter
#X_train = TimeSeriesResampler(sz=40).fit_transform(X_train)

km = TimeSeriesKMeans(n_clusters=3, verbose=True, metric="dtw", random_state=42, n_jobs=5)
y_pred = km.fit_predict(X_train)
y_pred

[01/02/23 14:19:39] WARNING  C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\utils\ ]8;id=954737;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py\warnings.py]8;;\:]8;id=40242;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py#109\109]8;;\
                             utils.py:149: DeprecationWarning:                                                     
                                                                                                                   
                             `np.float` is a deprecated alias for the builtin `float`. To silence                  
                             this warning, use `float` by itself. Doing this will not modify any                   
                             behavior and is safe. If you specifically wanted the numpy scalar                     
                             type, use `np.float64` here.                                                          
                             Deprecated in NumPy 1.20; for more details and guidance:                              
                             https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations                      
                                                                                                                   
                                                                                                                   

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 210372 out of 210372 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 631116 out of 631116 | elapsed:  8.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 631116 out of 631116 | elapsed:  8.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 631116 out of 631116 | elapsed:  8.6min finished


2888192364913.271 --> 

[01/02/23 14:59:06] WARNING  C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\baryce ]8;id=805365;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py\warnings.py]8;;\:]8;id=603264;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py#109\109]8;;\
                             nters\dba.py:599: ConvergenceWarning:                                                 
                                                                                                                   
                             DBA loss is increasing while it should not be. Stopping optimization.                 
                                                                                                                   
                                                                                                                   

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 631116 out of 631116 | elapsed:  8.1min finished


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 631116 out of 631116 | elapsed:  8.5min finished


1873040049062.329 --> 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 631116 out of 631116 | elapsed:  7.4min finished


1872721704525.356 --> 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\fabia\AppData\Local\Temp\ipykernel_40356\449604934.py:12 in <cell line: 12>             │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\fabia\\AppData\\Local\\Temp\\ipykernel_40356\\449604934.py'                          │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\clustering\kmeans.py:813 in      │
│ fit_predict                                                                                      │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\clustering\kmeans.py:780 in fit  │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\clustering\kmeans.py:665 in      │
│ _fit_one_init                                                                                    │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\clustering\kmeans.py:713 in      │
│ _update_centroids                                                                                │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\barycenters\dba.py:498 in        │
│ dtw_barycenter_averaging                                                                         │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\barycenters\dba.py:579 in        │
│ dtw_barycenter_averaging_one_init                                                                │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\utils\utils.py:209 in            │
│ to_time_series_dataset                                                                           │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\utils\utils.py:209 in <listcomp> │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\utils\utils.py:152 in            │
│ to_time_series                                                                                   │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\utils\utils.py:420 in ts_size    │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\utils\utils.py:146 in            │
│ to_time_series                                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
np.unique(y_pred, return_counts=True)

In [271]:
merged_data = catalog.load("merged_data")

[01/02/23 21:15:30] INFO     Loading data from 'merged_data' (ParquetDataSet)...                ]8;id=173887;file://C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=265399;file://C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

In [278]:
def to_timeseries(merged_data: pd.DataFrame) -> np.array:
    """ Convert merged data to timeseries for each WGMS_ID.

    Parameters
    ----------
    merged_data : pd.DataFrame
        Merged data.

    Returns
    -------
    np.array
        Array of Timeseries data.
    """
    features = ["AREA_CHANGE", "THICKNESS_CHG", "VOLUME_CHANGE", "ANNUAL_BALANCE", "AREA", "LENGTH", "MEDIAN_ELEVATION"]

    merged_data.dropna(subset=features, axis=0, how='all', inplace=True)

    x = merged_data.WGMS_ID.factorize()[0]
    y = merged_data.groupby(x).cumcount().values
    vals = merged_data.loc[:, features].values
    out_shp = (x.max() + 1, y.max() + 1, vals.shape[1])
    out = np.full(out_shp, np.nan)
    out[x, y] = vals

    timeseries = to_time_series_dataset(out)

    return timeseries

In [279]:
t = to_timeseries(merged_data)

<bound method NDFrame.head of         WGMS_ID   LATITUDE  LONGITUDE  PRIM_CLASSIFIC  FORM  FRONTAL_CHARS  \
0             0  79.514600 -90.876970             5.0   1.0            5.0   
1             0  79.514600 -90.876970             5.0   1.0            5.0   
2             0  79.514600 -90.876970             5.0   1.0            5.0   
3             0  79.514600 -90.876970             5.0   1.0            5.0   
4             1  79.441582 -90.957779             6.0   5.0            0.0   
...         ...        ...        ...             ...   ...            ...   
882326    35232        NaN        NaN             NaN   NaN            NaN   
882327    35232        NaN        NaN             NaN   NaN            NaN   
882328    35232        NaN        NaN             NaN   NaN            NaN   
882329    35232        NaN        NaN             NaN   NaN            NaN   
882330    35232        NaN        NaN             NaN   NaN            NaN   

        YEAR  AREA_CHANGE  THICKN

[01/02/23 21:18:24] WARNING  C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\utils\ ]8;id=923163;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py\warnings.py]8;;\:]8;id=6237;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py#109\109]8;;\
                             utils.py:149: DeprecationWarning:                                                     
                                                                                                                   
                             `np.float` is a deprecated alias for the builtin `float`. To silence                  
                             this warning, use `float` by itself. Doing this will not modify any                   
                             behavior and is safe. If you specifically wanted the numpy scalar                     
                             type, use `np.float64` here.                                                          
                             Deprecated in NumPy 1.20; for more details and guidance:                              
                             https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations                      
                                                                                                                   
                                                                                                                   

In [280]:
scaler = TimeSeriesScalerMeanVariance()
X = scaler.fit_transform(t)
km = TimeSeriesKMeans(
    n_clusters=3,
    verbose=False,
    metric="dtw",
    random_state=42,
    n_jobs=5
)
km.fit(X, None)

[01/02/23 21:19:19] WARNING  C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\utils\ ]8;id=838189;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py\warnings.py]8;;\:]8;id=298581;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py#109\109]8;;\
                             utils.py:149: DeprecationWarning:                                                     
                                                                                                                   
                             `np.float` is a deprecated alias for the builtin `float`. To silence                  
                             this warning, use `float` by itself. Doing this will not modify any                   
                             behavior and is safe. If you specifically wanted the numpy scalar                     
                             type, use `np.float64` here.                                                          
                             Deprecated in NumPy 1.20; for more details and guidance:                              
                             https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations                      
                                                                                                                   
                                                                                                                   

[01/02/23 21:20:12] WARNING  C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\utils\ ]8;id=608346;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py\warnings.py]8;;\:]8;id=182706;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py#109\109]8;;\
                             utils.py:149: DeprecationWarning:                                                     
                                                                                                                   
                             `np.float` is a deprecated alias for the builtin `float`. To silence                  
                             this warning, use `float` by itself. Doing this will not modify any                   
                             behavior and is safe. If you specifically wanted the numpy scalar                     
                             type, use `np.float64` here.                                                          
                             Deprecated in NumPy 1.20; for more details and guidance:                              
                             https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations                      
                                                                                                                   
                                                                                                                   

[01/02/23 21:21:03] WARNING  C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\prepro ]8;id=536960;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py\warnings.py]8;;\:]8;id=809806;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py#109\109]8;;\
                             cessing\preprocessing.py:289: RuntimeWarning:                                         
                                                                                                                   
                             Mean of empty slice                                                                   
                                                                                                                   
                                                                                                                   

[01/02/23 21:21:04] WARNING  C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\numpy\lib\nanf ]8;id=116738;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py\warnings.py]8;;\:]8;id=795512;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py#109\109]8;;\
                             unctions.py:1878: RuntimeWarning:                                                     
                                                                                                                   
                             Degrees of freedom <= 0 for slice.                                                    
                                                                                                                   
                                                                                                                   

                    WARNING  C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\utils\ ]8;id=904234;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py\warnings.py]8;;\:]8;id=532850;file://C:\Users\fabia\anaconda3\envs\glacier\lib\warnings.py#109\109]8;;\
                             utils.py:149: DeprecationWarning:                                                     
                                                                                                                   
                             `np.float` is a deprecated alias for the builtin `float`. To silence                  
                             this warning, use `float` by itself. Doing this will not modify any                   
                             behavior and is safe. If you specifically wanted the numpy scalar                     
                             type, use `np.float64` here.                                                          
                             Deprecated in NumPy 1.20; for more details and guidance:                              
                             https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations                      
                                                                                                                   
                                                                                                                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\fabia\AppData\Local\Temp\ipykernel_40356\2358582536.py:10 in <cell line: 10>            │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\fabia\\AppData\\Local\\Temp\\ipykernel_40356\\2358582536.py'                         │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\clustering\kmeans.py:780 in fit  │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\clustering\kmeans.py:649 in      │
│ _fit_one_init                                                                                    │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\clustering\kmeans.py:117 in      │
│ _k_init_metric                                                                                   │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\clustering\kmeans.py:638 in      │
│ metric_fun                                                                                       │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\metrics\dtw_variants.py:1224 in  │
│ cdist_dtw                                                                                        │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\tslearn\metrics\utils.py:78 in           │
│ _cdist_generic                                                                                   │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\joblib\parallel.py:1098 in __call__      │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\site-packages\joblib\parallel.py:975 in retrieve       │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\multiprocessing\pool.py:768 in get                     │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\multiprocessing\pool.py:765 in wait                    │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\threading.py:607 in wait                               │
│                                                                                                  │
│ C:\Users\fabia\anaconda3\envs\glacier\lib\threading.py:320 in wait                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt